In [1]:
import cv2
import dlib
import os
import glob

In [2]:
# Specify paths for new train and validation folders
new_train_folder = "new_train"
new_validation_folder = "new_validation"

# Create new train and validation folders if they don't exist
os.makedirs(new_train_folder, exist_ok=True)
os.makedirs(new_validation_folder, exist_ok=True)

In [3]:
# Load face detector and landmarks predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [11]:
def align_face(image, landmarks):
    # Define desired landmarks for alignment
    desired_landmarks = np.array([
        (0.315, 0.393),  # Left eye corner
        (0.685, 0.393),  # Right eye corner
        (0.500, 0.618),  # Nose tip
        (0.370, 0.800),  # Left mouth corner
        (0.630, 0.800)   # Right mouth corner
    ])
    
    # Convert landmarks to NumPy array and reshape for compatibility
    landmarks = np.array(landmarks).reshape(-1, 2)
    
    # Calculate transformation matrix using desired and detected landmarks
    aligned_image_size = (256, 256)
    transformation_matrix = cv2.estimateAffinePartial2D(
        landmarks, desired_landmarks * aligned_image_size[0])[0]
    
    # Apply transformation matrix to align the face
    aligned_image = cv2.warpAffine(
        image, transformation_matrix, aligned_image_size,
        flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    
    return aligned_image


In [12]:
# Process the training set
train_folder = "faceEmotionDetection/train"

In [13]:
for class_folder in os.listdir(train_folder):
    class_path = os.path.join(train_folder, class_folder)
    if not os.path.isdir(class_path):
        continue
    
    new_class_folder = os.path.join(new_train_folder, class_folder)
    os.makedirs(new_class_folder, exist_ok=True)
    
    image_files = glob.glob(os.path.join(class_path, "*.jpg"))
    for image_file in image_files:
        image = cv2.imread(image_file)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        faces = detector(gray)
        for face in faces:
            landmarks = predictor(gray, face)
            landmarks = [(landmark.x, landmark.y) for landmark in landmarks.parts()]
            aligned_face = align_face(image, landmarks)
            
            # Perform additional preprocessing (e.g., resizing, normalization, etc.) as needed
            
            filename = os.path.basename(image_file)
            new_filepath = os.path.join(new_class_folder, filename)
            
            cv2.imwrite(new_filepath, aligned_face)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\ptsetreg.cpp:1108: error: (-215:Assertion failed) count >= 0 && to.checkVector(2) == count in function 'cv::estimateAffinePartial2D'


In [10]:
# Process the validation set
validation_folder = "/content/drive/MyDrive/Kathak/faceEmotionDetection/validation"

In [ ]:
for class_folder in os.listdir(validation_folder):
    class_path = os.path.join(validation_folder, class_folder)
    if not os.path.isdir(class_path):
        continue
    
    new_class_folder = os.path.join(new_validation_folder, class_folder)
    os.makedirs(new_class_folder, exist_ok=True)
    
    image_files = glob.glob(os.path.join(class_path, "*.jpg"))
    for image_file in image_files:
        image = cv2.imread(image_file)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        faces = detector(gray)
        for face in faces:
            landmarks = predictor(gray, face)
            landmarks = [(landmark.x, landmark.y) for landmark in landmarks.parts()]
            aligned_face = align_face(image, landmarks)

            # Perform additional preprocessing (e.g., resizing, normalization, etc.) as needed

            filename = os.path.basename(image_file)
            new_filepath = os.path.join(new_class_folder, filename)

            cv2.imwrite(new_filepath, aligned_face)